In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from kmodes.kmodes import KModes
import functions

sns.set_style('whitegrid')

In [2]:
data_full = pd.read_csv("./data/generated/Analizsiz_v1.3.csv")

In [3]:
data = data_full.sample(1000)

In [4]:
def apply_worker_type(row):
    try:
        return row["newpositiongroupcode"][0]
    except:
        return None
data['worker_type'] = data.apply(apply_worker_type,axis=1)

def apply_level(row):
    try:
        level = row["newpositiongroupcode"][1]
        if level == "O":
            return 0
        return int(level)
    except:
        return -1
    
data['expertise'] = data.apply(apply_level,axis=1)
data

,Unnamed: 0,classification,positionId,positiondescription,newpositiongroupcode,companyId,SalaryType,leavedate,salarycount,creationdate,Price,mainsectorname,worker_type,expertise
69046,69046,İnşaat,791,İç Mimar,B2,-1,Brut,20180101.0,12,2018-09-04 12:48:23.184,2100,NaN,B,2
289342,289342,Hizmet,226,Garson,M1,267879,Net,NaN,15,2020-07-16 19:03:12.731,3400,Eğlence - Kültür - Sanat,M,1
396108,396108,Finans,1321,Muhasebe Elemanı,B1,430830,Net,NaN,12,2019-10-10 09:49:33.706,3500,NaN,B,1
130676,130676,Bilgi Teknolojileri,3193,Yazılım Geliştirme Uzmanı,B2,272757,Net,NaN,12,2019-11-28 15:50:20.512,6000,Danışmanlık,B,2
237540,237540,Satış - Pazarlama,59,Aktif Pazarlama Yetkili Yardımcısı,B3,260748,Net,NaN,16,2019-10-02 13:10:16.059,3000,Finans - Ekonomi,B,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440499,440499,Üretim,1940,Üretim Planlama Şefi,B3,344707,Net,20200101.0,12,2020-08-11 22:57:09.526,3400,Bilişim,B,3
253323,253323,Makine,3602,Makine İkmal ve Bakım Şefi,B3,301026,Net,NaN,12,2019-12-04 15:41:34.390,2900,Yapı,B,3
77189,77189,Makine,3584,Makine Operatör Yardımcısı,M1,288962,Net,NaN,12,2019-10-03 15:07:23.360,2020,Üretim / Endüstriyel Ürünler,M,1
466278,466278,Depolama / Dağıtım,119,Depo Görevlisi,B1,286885,Net,NaN,15,2021-05-02 09:40:56.226,4000,Ticaret,B,1


In [5]:
data = data[["classification", "positiondescription", "mainsectorname","companyId","worker_type","expertise"]]
data.rename(
    columns = {
        'classification':'position_sector', 
        'positiondescription':'position_name',
        "mainsectorname": "company_sector",
        "companyId": "company_id"
    }, 
    inplace = True
)
data = data.dropna()

data = data.drop(["company_id"],axis=1)

data

/tmp/ipykernel_15133/900268052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(


,position_sector,position_name,company_sector,worker_type,expertise
289342,Hizmet,Garson,Eğlence - Kültür - Sanat,M,1
130676,Bilgi Teknolojileri,Yazılım Geliştirme Uzmanı,Danışmanlık,B,2
237540,Satış - Pazarlama,Aktif Pazarlama Yetkili Yardımcısı,Finans - Ekonomi,B,3
358406,Basın Medya,Junior Art Direktör,Medya,B,4
398437,Güvenlik,Güvenlik Müdürü,Yapı,B,4
...,...,...,...,...,...
440499,Üretim,Üretim Planlama Şefi,Bilişim,B,3
253323,Makine,Makine İkmal ve Bakım Şefi,Yapı,B,3
77189,Makine,Makine Operatör Yardımcısı,Üretim / Endüstriyel Ürünler,M,1
466278,Depolama / Dağıtım,Depo Görevlisi,Ticaret,B,1


In [6]:
from sklearn.metrics import silhouette_samples
from matplotlib import cm

def plot_silhouette(distance_matrix,clusters):
    cluster_labels = np.unique(clusters) # kume etiketlerini cekelim

    print('Labels')
    print(cluster_labels)

    print('Shape')
    print(cluster_labels.shape)

    n_clusters = cluster_labels.shape[0]  

    print('Kume sayisi')
    print(n_clusters)

    silhouette_vals = silhouette_samples(distance_matrix, clusters, metric='precomputed')

    #print('silhouette vals:')
    #print(silhouette_vals)

    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    for i, c in enumerate(cluster_labels):  # her bir kume icin 0,1,2 ...
        c_silhouette_vals = silhouette_vals[clusters == c] # su anki küme için
        c_silhouette_vals.sort() # değerleri sırala
        y_ax_upper += len(c_silhouette_vals) 
        color = cm.jet(float(i) / n_clusters) # bir renk sec
        plt.barh(range(y_ax_lower, y_ax_upper),
                c_silhouette_vals,
                height=1.0,
                edgecolor='none',
                color=color)
        yticks.append((y_ax_lower + y_ax_upper) / 2.)
        y_ax_lower += len(c_silhouette_vals)
    silhouette_avg = np.mean(silhouette_vals)


    print(silhouette_avg)

    plt.axvline(silhouette_avg, color="red", linestyle="--")
    plt.yticks(yticks, cluster_labels + 1)   # 0.ci yazmasin, 1den baslasin
    plt.ylabel('Küme')
    plt.xlabel('Silhouette katsayısı')
    plt.show()


In [7]:
import rock
clusters = rock.rock(data,eps=0.6,max_clusters=5)
clusters

[[  0]
 [  1]
 [  2]
 [  3]
 [  4]
 [  5]
 [  6]
 [  7]
 [  8]
 [  9]
 [ 10]
 [ 11]
 [ 12]
 [ 13]
 [ 14]
 [ 15]
 [ 16]
 [ 17]
 [ 18]
 [ 19]
 [ 20]
 [ 21]
 [ 22]
 [ 23]
 [ 24]
 [ 25]
 [ 26]
 [ 27]
 [ 28]
 [ 29]
 [ 30]
 [ 31]
 [ 32]
 [ 33]
 [ 34]
 [ 35]
 [ 36]
 [ 37]
 [ 38]
 [ 39]
 [ 40]
 [ 41]
 [ 42]
 [ 43]
 [ 44]
 [ 45]
 [ 46]
 [ 47]
 [ 48]
 [ 49]
 [ 50]
 [ 51]
 [ 52]
 [ 53]
 [ 54]
 [ 55]
 [ 56]
 [ 57]
 [ 58]
 [ 59]
 [ 60]
 [ 61]
 [ 62]
 [ 63]
 [ 64]
 [ 65]
 [ 66]
 [ 67]
 [ 68]
 [ 69]
 [ 70]
 [ 71]
 [ 72]
 [ 73]
 [ 74]
 [ 75]
 [ 76]
 [ 77]
 [ 78]
 [ 79]
 [ 80]
 [ 81]
 [ 82]
 [ 83]
 [ 84]
 [ 85]
 [ 86]
 [ 87]
 [ 88]
 [ 89]
 [ 90]
 [ 91]
 [ 92]
 [ 93]
 [ 94]
 [ 95]
 [ 96]
 [ 97]
 [ 98]
 [ 99]
 [100]
 [101]
 [102]
 [103]
 [104]
 [105]
 [106]
 [107]
 [108]
 [109]
 [110]
 [111]
 [112]
 [113]
 [114]
 [115]
 [116]
 [117]
 [118]
 [119]
 [120]
 [121]
 [122]
 [123]
 [124]
 [125]
 [126]
 [127]
 [128]
 [129]
 [130]
 [131]
 [132]
 [133]
 [134]
 [135]
 [136]
 [137]
 [138]
 [139]
 [140]
 [141]
 [142]

TypeError: 'numpy.int64' object is not iterable

In [ ]:
len(clusters)